<a href="https://colab.research.google.com/github/SaqlainHussainShah/implementation-of-sdi/blob/master/Deep_Learning_Separate_Notebook_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Steps for implementation
#### Reading CSV
#### Data Description
#### Data Cleaning
#### Data Normalization
#### Features
#### Applying KNN and NB
#### Confusion Matrix

In [0]:
#Importing Necessary Modules
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.metrics import accuracy_score

### Reading CSV

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df1=pd.read_csv("/content/drive/My Drive/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df2=pd.read_csv("/content/drive/My Drive/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
df3=pd.read_csv("/content/drive/My Drive/Friday-WorkingHours-Morning.pcap_ISCX.csv")
df4=pd.read_csv("/content/drive/My Drive/Monday-WorkingHours.pcap_ISCX.csv")
df5=pd.read_csv("/content/drive/My Drive/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
df6=pd.read_csv("/content/drive/My Drive/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
df7=pd.read_csv("/content/drive/My Drive/Tuesday-WorkingHours.pcap_ISCX.csv")
df8=pd.read_csv("/content/drive/My Drive/Wednesday-workingHours.pcap_ISCX.csv")


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8])

In [0]:
del df1,df2,df3,df4,df5,df6,df7,df8


In [0]:
# category B attacks
df_brute_force=df[df[' Label']=='Web Attack-Brute Force']
df_xss=df[df[' Label']=='Web Attack-XSS']
df_sql=df[df[' Label']=='Web Attack-Sql Injection']
#df_heartbleed=df[df[' Label']=='Heartbleed']
df_slowloris=df[df[' Label']=='DoS slowloris']
df_portscan=df[df[' Label']=='PortScan']
df_slow_http_test=df[df[' Label']=='DoS Slowhttptest']

In [0]:
# benign data
df_benign=df[df[' Label']=='BENIGN']

In [8]:
df_cat_B=pd.concat([df_brute_force,df_xss,df_sql,df_slowloris,df_portscan,df_slow_http_test, df_benign])
df_cat_B.groupby(' Label').first()


,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,...,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
Label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
BENIGN,192.168.10.5-104.16.207.165-54865-443-6,104.16.207.165,443,192.168.10.5,54865,6,7/7/2017 3:30,3,2,0,12.0,0.0,6.0,6.0,6.000000,0.000000,0.0,0.0,0.000000,0.000000,4000000,666666.6667,3.000000e+00,0.000000e+00,3.0,3.0,3.0,3.000000e+00,0.000000e+00,3.0,3.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,...,6.0,6.0,6.000000,0.000000,0.00000,0,0,0,0,1,0,0,0,0.0,9.000000,6.000000,0.000000,40,0,0,0,0,0,0,2,12,0,0,33,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DoS Slowhttptest,172.16.0.1-192.168.10.50-33372-80-6,172.16.0.1,33372,192.168.10.50,80,6,5/7/2017 10:15,16045,5,3,356.0,1964.0,356.0,0.0,71.200000,159.208040,1964.0,0.0,654.666667,1133.915929,144593.3313,498.597694,2.292143e+03,2.236413e+03,5415.0,73.0,16045.0,4.011250e+03,1.779768e+03,5415.0,1459.0,9788.0,4.894000e+03,9.461089e+02,5563.0,4225.0,0,0,0,0,...,0.0,1964.0,257.777778,650.575472,423248.44440,0,0,0,1,0,0,0,0,0.0,290.000000,71.200000,654.666667,168,0,0,0,0,0,0,5,356,3,1964,29200,235,1,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DoS slowloris,172.16.0.1-192.168.10.50-49631-80-6,172.16.0.1,49631,192.168.10.50,80,6,5/7/2017 2:24,5169956,8,6,1101.0,4222.0,410.0,0.0,137.625000,185.758628,3525.0,0.0,703.666667,1395.868284,1029.602573,2.707953414,3.976889e+05,1.368409e+06,4951173.0,112.0,218783.0,3.125471e+04,3.068629e+04,78311.0,219.0,5169833.0,1.033967e+06,2.218975e+06,5002421.0,840.0,0,0,0,0,...,0.0,3525.0,354.866667,895.115146,801231.12380,0,0,0,1,0,0,0,0,0.0,380.214286,137.625000,703.666667,172,0,0,0,0,0,0,8,1101,6,4222,8192,254,7,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PortScan,172.16.0.1-192.168.10.50-40620-80-6,172.16.0.1,40620,192.168.10.50,80,6,7/7/2017 1:05,5021059,6,5,703.0,1414.0,356.0,0.0,117.166667,181.536130,1050.0,0.0,282.800000,456.923626,421.6242032,2.190772903,5.021059e+05,1.568379e+06,4965658.0,19.0,55401.0,1.108020e+04,1.761267e+04,41863.0,19.0,5020928.0,1.255232e+06,2.499940e+06,5005133.0,1053.0,0,0,0,0,...,0.0,1050.0,176.416667,317.471103,100787.90150,0,0,0,1,0,0,0,0,0.0,192.454545,117.166667,282.800000,200,0,0,0,0,0,0,6,703,5,1414,29200,243,2,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Web Attack-Brute Force,172.16.0.1-192.168.10.50-44380-80-6,172.16.0.1,44380,192.168.10.50,80,6,6/7/2017 9:15,5185118,7,7,1022.0,2321.0,372.0,0.0,146.000000,184.078788,1047.0,0.0,331.571429,439.659284,644.7297824,2.700034985,3.988552e+05,1.372181e+06,4963956.0,4.0,221162.0,3.686033e+04,5.614102e+04,141434.0,4.0,5185004.0,8.641673e+05,2.027593e+06,5001548.0,879.0,0,0,0,0,...,0.0,1047.0,222.866667,331.323939,109775.55240,0,0,0,1,0,0,0,0,1.0,238.785714,146.000000,331.571429,232,0,0,0,0,0,0,7,1022,7,2321,29200,252,3,32,0.0,0.0,0

In [0]:
del df_brute_force,df_xss,df_sql,df_slowloris,df_portscan,df_slow_http_test, df_benign

In [0]:
df=df_cat_B.copy()

In [0]:
del df_cat_B

In [12]:
df.groupby(' Label').first()

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,...,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
Label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
BENIGN,192.168.10.5-104.16.207.165-54865-443-6,104.16.207.165,443,192.168.10.5,54865,6,7/7/2017 3:30,3,2,0,12.0,0.0,6.0,6.0,6.000000,0.000000,0.0,0.0,0.000000,0.000000,4000000,666666.6667,3.000000e+00,0.000000e+00,3.0,3.0,3.0,3.000000e+00,0.000000e+00,3.0,3.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,...,6.0,6.0,6.000000,0.000000,0.00000,0,0,0,0,1,0,0,0,0.0,9.000000,6.000000,0.000000,40,0,0,0,0,0,0,2,12,0,0,33,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DoS Slowhttptest,172.16.0.1-192.168.10.50-33372-80-6,172.16.0.1,33372,192.168.10.50,80,6,5/7/2017 10:15,16045,5,3,356.0,1964.0,356.0,0.0,71.200000,159.208040,1964.0,0.0,654.666667,1133.915929,144593.3313,498.597694,2.292143e+03,2.236413e+03,5415.0,73.0,16045.0,4.011250e+03,1.779768e+03,5415.0,1459.0,9788.0,4.894000e+03,9.461089e+02,5563.0,4225.0,0,0,0,0,...,0.0,1964.0,257.777778,650.575472,423248.44440,0,0,0,1,0,0,0,0,0.0,290.000000,71.200000,654.666667,168,0,0,0,0,0,0,5,356,3,1964,29200,235,1,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DoS slowloris,172.16.0.1-192.168.10.50-49631-80-6,172.16.0.1,49631,192.168.10.50,80,6,5/7/2017 2:24,5169956,8,6,1101.0,4222.0,410.0,0.0,137.625000,185.758628,3525.0,0.0,703.666667,1395.868284,1029.602573,2.707953414,3.976889e+05,1.368409e+06,4951173.0,112.0,218783.0,3.125471e+04,3.068629e+04,78311.0,219.0,5169833.0,1.033967e+06,2.218975e+06,5002421.0,840.0,0,0,0,0,...,0.0,3525.0,354.866667,895.115146,801231.12380,0,0,0,1,0,0,0,0,0.0,380.214286,137.625000,703.666667,172,0,0,0,0,0,0,8,1101,6,4222,8192,254,7,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PortScan,172.16.0.1-192.168.10.50-40620-80-6,172.16.0.1,40620,192.168.10.50,80,6,7/7/2017 1:05,5021059,6,5,703.0,1414.0,356.0,0.0,117.166667,181.536130,1050.0,0.0,282.800000,456.923626,421.6242032,2.190772903,5.021059e+05,1.568379e+06,4965658.0,19.0,55401.0,1.108020e+04,1.761267e+04,41863.0,19.0,5020928.0,1.255232e+06,2.499940e+06,5005133.0,1053.0,0,0,0,0,...,0.0,1050.0,176.416667,317.471103,100787.90150,0,0,0,1,0,0,0,0,0.0,192.454545,117.166667,282.800000,200,0,0,0,0,0,0,6,703,5,1414,29200,243,2,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Web Attack-Brute Force,172.16.0.1-192.168.10.50-44380-80-6,172.16.0.1,44380,192.168.10.50,80,6,6/7/2017 9:15,5185118,7,7,1022.0,2321.0,372.0,0.0,146.000000,184.078788,1047.0,0.0,331.571429,439.659284,644.7297824,2.700034985,3.988552e+05,1.372181e+06,4963956.0,4.0,221162.0,3.686033e+04,5.614102e+04,141434.0,4.0,5185004.0,8.641673e+05,2.027593e+06,5001548.0,879.0,0,0,0,0,...,0.0,1047.0,222.866667,331.323939,109775.55240,0,0,0,1,0,0,0,0,1.0,238.785714,146.000000,331.571429,232,0,0,0,0,0,0,7,1022,7,2321,29200,252,3,32,0.0,0.0,0

In [0]:
# remove some data
df=df.iloc[0:-800000]

### Data Description

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1645502 entries, 12637 to 80025
Data columns (total 85 columns):
Flow ID                         1645502 non-null object
 Source IP                      1645502 non-null object
 Source Port                    1645502 non-null int64
 Destination IP                 1645502 non-null object
 Destination Port               1645502 non-null int64
 Protocol                       1645502 non-null int64
 Timestamp                      1645502 non-null object
 Flow Duration                  1645502 non-null int64
 Total Fwd Packets              1645502 non-null int64
 Total Backward Packets         1645502 non-null int64
Total Length of Fwd Packets     1645502 non-null float64
 Total Length of Bwd Packets    1645502 non-null float64
 Fwd Packet Length Max          1645502 non-null float64
 Fwd Packet Length Min          1645502 non-null float64
 Fwd Packet Length Mean         1645502 non-null float64
 Fwd Packet Length Std          1645502 non-nu

In [15]:
df.describe()

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1645502.0,1.645502e+06,1645502.0,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1645502.0,1645502.0,1645502.0,1645502.0,1645502.0,1645502.0,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06,1.645502e+06
mean,4.044689e+04,9.302189e+03,1.014187e+01,1.070110e+07,1.027427e+01,1.173266e+01,5.935790e+02,1.850419e+04,2.232141e+02,2.126460e+01,6.420241e+01,7.363384e+01,3.532719e+02,4.427942e+01,1.426041e+02,1.111276e+02,9.405868e+05,1.600199e+06,4.469137e+06,1.549574e+05,1.037981e+07,1.956006e+06,1.227063e+06,4.313119e+06,1.210055e+06,9.577729e+06,1.846212e+06,9.841588e+05,3.724377e+06,1.189821e+06,5.059672e-02,0.0,1.914309e-04,0.0,-3.377753e+04,-2.105793e+03,5.781747e+04,8.849170e+03,1.770790e+01,4.557611e+02,9.881104e+01,1.371538e+02,1.147347e+05,1.507443e-02,5.059672e-02,1.956850e-04,3.513159e-01,2.596284e-01,1.050774e-01,1.914309e-04,1.962927e-04,7.300988e-01,1.122122e+02,6.420241e+01,1.426041e+02,-3.377753e+04,0.0,0.0,0.0,0.0,0.0,0.0,1.027427e+01,5.935610e+02,1.173266e+01,1.850422e+04,8.040349e+03,2.171940e+03,6.040525e+00,-2.777508e+03,8.924682e+04,4.857258e+04,1.706223e+05,6.133850e+04,3.664827e+06,3.359839e+05,3.939331e+06,3.381001e+06
std,2.320548e+04,1.921364e+04,5.336197e+00,2.936737e+07,8.516888e+02,1.136577e+03,1.152103e+04,2.577212e+06,8.234829e+02,7.310018e+01,2.146965e+02,3.307458e+02,7.680580e+02,6.969934e+01,2.693177e+02,2.625537e+02,4.275026e+06,6.080552e+06,1.464755e+07,2.745539e+06,2.924360e+07,9.655868e+06,4.763988e+06,1.477134e+07,9.345239e+06,2.835073e+07,9.664011e+06,4.372126e+06,1.397375e+07,9.241125e+06,2.191728e-01,0.0,1.383454e-02,0.0,2.659175e+07,1.235244e+06,2.249010e+05,4.231218e+04,2.477148e+01,1.069845e+03,1.812941e+02,3.097190e+02,6.344723e+05,1.218491e-01,2.191728e-01,1.398738e-02,4.773816e-01,4.384308e-01,3.066532e-01,1.383454e-02,1.400908e-02,7.375904e-01,1.922399e+02,2.146965e+02,2.693177e+02,2.659175e+07,0.0,0.0,0.0,0.0,0.0,0.0,8.516888e+02,1.15

In [16]:
df.columns

Index(['Flow ID', ' Source IP', ' Source Port', ' Destination IP',
       ' Destination Port', ' Protocol', ' Timestamp', ' Flow Duration',
       ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Length', ' Max Pa

### Data Cleaning

In [17]:
df2=df[df.columns[7:-1]]
df2[' Label']=df[' Label']
df=df2
df2=[]
del df2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
len(df.columns)

78

In [0]:
df=df.dropna( axis=0, how='any')
df=df.replace(',,', np.nan, inplace=False)
df=df.drop(columns=[' Fwd Header Length.1'], axis=1, inplace=False)

In [0]:
df.replace("Infinity", 0, inplace=True)
df['Flow Bytes/s'].replace("Infinity", 0,inplace=True)
df[" Flow Packets/s"].replace("Infinity", 0, inplace=True)
df[" Flow Packets/s"].replace(np.nan, 0, inplace=True)
df['Flow Bytes/s'].replace(np.nan, 0,inplace=True)


df["Bwd Avg Bulk Rate"].replace("Infinity", 0, inplace=True)
df["Bwd Avg Bulk Rate"].replace(",,", 0, inplace=True)
df["Bwd Avg Bulk Rate"].replace(np.nan, 0, inplace=True)

df[" Bwd Avg Packets/Bulk"].replace("Infinity", 0, inplace=True)
df[" Bwd Avg Packets/Bulk"].replace(",,", 0, inplace=True)
df[" Bwd Avg Packets/Bulk"].replace(np.nan, 0, inplace=True)


df[" Bwd Avg Bytes/Bulk"].replace("Infinity", 0, inplace=True)
df[" Bwd Avg Bytes/Bulk"].replace(",,", 0, inplace=True)
df[" Bwd Avg Bytes/Bulk"].replace(np.nan, 0, inplace=True)


df[" Fwd Avg Bulk Rate"].replace("Infinity", 0, inplace=True)
df[" Fwd Avg Bulk Rate"].replace(",,", 0, inplace=True)
df[" Fwd Avg Bulk Rate"].replace(np.nan, 0, inplace=True)


df[" Fwd Avg Packets/Bulk"].replace("Infinity", 0, inplace=True)
df[" Fwd Avg Packets/Bulk"].replace(",,", 0, inplace=True)
df[" Fwd Avg Packets/Bulk"].replace(np.nan, 0, inplace=True)


df["Fwd Avg Bytes/Bulk"].replace("Infinity", 0, inplace=True)
df["Fwd Avg Bytes/Bulk"].replace(",,", 0, inplace=True)
df["Fwd Avg Bytes/Bulk"].replace(np.nan, 0, inplace=True)


df[" CWE Flag Count"].replace("Infinity", 0, inplace=True)
df[" CWE Flag Count"].replace(",,", 0, inplace=True)
df[" CWE Flag Count"].replace(np.nan, 0, inplace=True)

df[" Bwd URG Flags"].replace("Infinity", 0, inplace=True)
df[" Bwd URG Flags"].replace(",,", 0, inplace=True)
df[" Bwd URG Flags"].replace(np.nan, 0, inplace=True)

df[" Bwd PSH Flags"].replace("Infinity", 0, inplace=True)
df[" Bwd PSH Flags"].replace(",,", 0, inplace=True)
df[" Bwd PSH Flags"].replace(np.nan, 0, inplace=True)

df[" Fwd URG Flags"].replace("Infinity", 0, inplace=True)
df[" Fwd URG Flags"].replace(",,", 0, inplace=True)
df[" Fwd URG Flags"].replace(np.nan, 0, inplace=True)

In [0]:
df["Flow Bytes/s"]=df["Flow Bytes/s"].astype("float64")
df[' Flow Packets/s']=df[" Flow Packets/s"].astype("float64")

In [0]:
df['Bwd Avg Bulk Rate']=df["Bwd Avg Bulk Rate"].astype("float64")
df[' Bwd Avg Packets/Bulk']=df[" Bwd Avg Packets/Bulk"].astype("float64")
df[' Bwd Avg Bytes/Bulk']=df[" Bwd Avg Bytes/Bulk"].astype("float64")
df[' Fwd Avg Bulk Rate']=df[" Fwd Avg Bulk Rate"].astype("float64")
df[' Fwd Avg Packets/Bulk']=df[" Fwd Avg Packets/Bulk"].astype("float64")
df['Fwd Avg Bytes/Bulk']=df["Fwd Avg Bytes/Bulk"].astype("float64")
df[' CWE Flag Count']=df[" CWE Flag Count"].astype("float64")
df[' Bwd URG Flags']=df[" Bwd URG Flags"].astype("float64")
df[' Bwd PSH Flags']=df[" Bwd PSH Flags"].astype("float64")
df[' Fwd URG Flags']=df[" Fwd URG Flags"].astype("float64")

In [0]:
pd.set_option('display.max_rows', df.shape[0])

In [0]:
df.replace('Infinity',0.0, inplace=True)

In [0]:
df.replace('NaN',0.0, inplace=True)

In [26]:
df.columns

Index([' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length',
       ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance',
       'FIN Flag Count', ' SY

In [0]:


X=df[df.columns[0:-1]]
y=df[df.columns[-1]]

del df

### Data Normalization

In [0]:
from scipy import stats

In [29]:


cols = list(X.columns)
for col in cols:
    X[col] = stats.zscore(X[col])



/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:2315: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


In [30]:
X.head()

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
12637,-0.187859,-0.003845,-0.004165,0.037178,-0.006280,0.180638,-0.290915,0.380938,0.333883,0.903131,-0.635343,0.701563,1.251187,-0.058956,-0.284051,-0.126738,-0.037528,0.033745,-0.056441,-0.347403,-0.198767,-0.245802,-0.282436,-0.129491,-0.154973,-0.101635,0.238617,0.091363,-0.128665,-0.230867,NaN,-0.013838,NaN,0.001279,0.001893,-0.257089,-0.209121,-0.714912,0.552565,0.684189,0.626844,-0.007836,-0.123721,-0.230867,-0.013991,1.358718,-0.592046,-0.342641,-0.013838,-0.014013,0.365884,0.658323,0.380938,0.701563,NaN,NaN,NaN,NaN,NaN,NaN,-0.003845,0.037245,-0.004165,-0.006280,1.372278,-0.216123,-0.004383,0.002847,-0.125793,-0.113961,-0.155162,-0.096887,-0.273255,-0.100212,-0.276465,-0.259042
12643,-0.192209,-0.000323,-0.004165,0.012355,-0.003347,0.208567,-0.290915,0.043735,0.248036,5.195583,-0.635343,4.711002,5.822443,-0.058902,-0.284049,-0.146098,-0.059387,0.033688,-0.056430,-0.182035,-0.144391,0.088855,0.043969,-0.129471,-0.335870,-0.190082,-0.222695,-0.264996,-0.128756,-0.230867,NaN,-0.013838,NaN,0.001283,0.001893,-0.257087,-0.209120,-0.714912,3.634177,2.707953,3.400620,2.052669,-0.123721,-0.230867,-0.013991,1.358718,-0.592046,-0.342641,-0.013838,-0.014013,-0.989845,2.664517,0.043735,4.711002,NaN,NaN,NaN,NaN,NaN,NaN,-0.000323,0.012378,-0.004165,-0.003347,1.372278,-0.217137,-0.005824,0.002847,-0.125793,-0.113961,-0.155162,-0.096887,-0.273255,-0.100212,-0.276465,-0.259042
12700,-0.364408,-0.010890,-0.009444,-0.051524,-0.007180,-0.271078,-0.290915,-0.299057,-0.222643,-0.459987,-0.635343,-0.529540,-0.423285,-0.058980,-0.178720,-0.220013,-0.263183,-0.305125,-0.056413,-0.354966,-0.202584,-0.257586,-0.292011,-0.129491,-0.337852,-0.191051,-0.225112,-0.266543,-0.128760,-0.230867,NaN,-0.013838,NaN,0.001272,0.001731,-0.202204,0.082608,-0.714912,-0.426036,-0.545072,-0.442863,-0.180846,-0.123721,-0.230867,-0.013991,-0.735988,1.689057,2.918509,-0.013838,-0.014013,0.365884,-0.583754,-0.299057,-0.529540,NaN,NaN,NaN,NaN,NaN,NaN,-0.010890,-0.051613,-0.009444,-0.007180,-0.503893,-0.216123,-0.008707,0.002847,-0.125793,-0.113961,-0.155162,-0.096887,-0.273255,-0.100212,-0.276465,-0.259042
12712,-0.184931,-0.003845,-0.005924,0.032578,-0.006626,0.410139,-0.290915,0.345674,0.504469,0.927867,-0.635343,0.532359,1.342543,-0.058963,-0.284053,-0.107947,-0.018343,0.033606,-0.056426,-0.174726,-0.111606,0.162859,0.043887,-0.129457,-0.328321,-0.184062,-0.195013,-0.247583,-0.128649,-0.230867,NaN,-0.013838,NaN,0.001279,0.001841,-0.257089,-0.209130,-0.714912,0.570324,0.472783,0.622786,-0.009147,-0.123721,-0.230867,-0.013991,1.358718,-0.592046,-0.342641,-0.013838,-0.014013,-0.989845,0.456141,0.345674,0.532359,NaN,NaN,NaN,NaN,NaN,NaN,-0.003845,0.032637,-0.005924,-0.006625,1.372278,-0.217024,-0.005824,0.002847,-0.125793,-0.11

### Feature Selection

In [0]:
features=[" Fwd Packet Length Max"," Flow IAT Std"," Fwd Packet Length Std" ,"Fwd IAT Total",' Flow Packets/s',' Flow Duration', " Fwd Packet Length Mean", " Total Length of Bwd Packets", "Flow Bytes/s", "Total Length of Fwd Packets", " Flow IAT Mean", " Bwd Packet Length Mean",  " Flow IAT Max", " Bwd Packet Length Std", ' Total Fwd Packets', ' Total Backward Packets']

In [32]:
X=X[features].copy()
X.head()

,Fwd Packet Length Max,Flow IAT Std,Fwd Packet Length Std,Fwd IAT Total,Flow Packets/s,Flow Duration,Fwd Packet Length Mean,Total Length of Bwd Packets,Flow Bytes/s,Total Length of Fwd Packets,Flow IAT Mean,Bwd Packet Length Mean,Flow IAT Max,Bwd Packet Length Std,Total Fwd Packets,Total Backward Packets
12637,0.180638,-0.037528,0.333883,-0.347403,-0.284051,-0.187859,0.380938,-0.006280,-0.058956,0.037178,-0.126738,0.701563,0.033745,1.251187,-0.003845,-0.004165
12643,0.208567,-0.059387,0.248036,-0.182035,-0.284049,-0.192209,0.043735,-0.003347,-0.058902,0.012355,-0.146098,4.711002,0.033688,5.822443,-0.000323,-0.004165
12700,-0.271078,-0.263183,-0.222643,-0.354966,-0.178720,-0.364408,-0.299057,-0.007180,-0.058980,-0.051524,-0.220013,-0.529540,-0.305125,-0.423285,-0.010890,-0.009444
12712,0.410139,-0.018343,0.504469,-0.174726,-0.284053,-0.184931,0.345674,-0.006626,-0.058963,0.032578,-0.107947,0.532359,0.033606,1.342543,-0.003845,-0.005924
12791,0.414996,-0.005434,0.508016,-0.183291,-0.284054,-0.193460,0.348335,-0.006626,-0.058962,0.032925,-0.102597,0.797834,0.033656,1.490657,-0.003845,-0.006804


In [33]:
len(X.columns)

16

### Applying Algorithms

In [0]:
from sklearn.model_selection import train_test_split     # import module for train test split

In [0]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2, random_state=10)

#### KNN

In [0]:
from sklearn.neighbors import KNeighborsClassifier

In [0]:
t1=time.time()

for i in range(1,len(X_train.columns)+1):
    knn=KNeighborsClassifier(n_neighbors=i)
    model_knn=knn.fit(X_train,y_train)
    yhat=model_knn.predict(X_test)
    print("for " , i,  " as K, accuracy is : ", accuracy_score(y_test, yhat))
t2=time.time()
print(" time for ", i ," k's :: ", (t2-t1)/60 , " minutes")

for  1  as K, accuracy is :  0.998781233449007
for  2  as K, accuracy is :  0.9987035268568297
for  3  as K, accuracy is :  0.9988364460276594
for  4  as K, accuracy is :  0.998807817283173
for  5  as K, accuracy is :  0.9988323562070185
for  6  as K, accuracy is :  0.9988200867450958
for  7  as K, accuracy is :  0.9988405358483003
for  8  as K, accuracy is :  0.9988303112966981
for  9  as K, accuracy is :  0.9988364460276594
for  10  as K, accuracy is :  0.9988180418347753
for  11  as K, accuracy is :  0.9987996376418912
for  12  as K, accuracy is :  0.998785323269648
for  13  as K, accuracy is :  0.9987607843458025
for  14  as K, accuracy is :  0.9987607843458025
for  15  as K, accuracy is :  0.9987260208703548
for  16  as K, accuracy is :  0.9987260208703548
 time for  16  k's ::  61.21009658972422  minutes


In [0]:
import pickle
filename = 'drive/My Drive/model_cat_B.sav'

In [0]:
pickle.dump(model_knn, open(filename, 'wb'))

In [0]:
# to load model
model_knn = pickle.load(open(filename, 'rb'))
y_hat=model_knn.predict(X_test)

#### Naive Bayes

##### Gaussian Naive Bayes

In [0]:
from sklearn.naive_bayes import GaussianNB

In [0]:
t1=time.time()

gnb = GaussianNB()
model_gnb = gnb.fit(X_train, y_train)
yhat_gnb=model_gnb.predict(X_test)

print("Accuracy :: ", accuracy_score(y_test,yhat))

t2=time.time()
print(" time for Naive Baye's :: ", (t2-t1) , " seconds")

Accuracy ::  0.9987260208703548
 time for Naive Baye's ::  5.13779616355896  seconds


##### Bernoulli Naive Bayes

In [0]:
from sklearn.naive_bayes import BernoulliNB

In [0]:


clf = BernoulliNB()
clf.fit(X_train, y_train)
yhat_bnb=clf.predict(X_test)
print("Accuracy ::: ", accuracy_score(y_test,yhat_bnb))



Accuracy :::  0.8611669485234725


### Confusion Matrix

In [0]:
y_test_arr=y_test.as_matrix()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [0]:
true={}
false={}
not_detected={}

for x in range(len(y_test_arr)):
    if y_test_arr[x]==yhat[x]:
      if y_test_arr[x] in true:
        true[y_test_arr[x]]=true[y_test_arr[x]]+1
      else:
        true[y_test_arr[x]]=1
    elif y_test_arr[x]!=yhat[x]:
      if yhat[x] in false:
        false[yhat[x]]=false[yhat[x]]+1

        if y_test_arr[x] in not_detected:
          not_detected[y_test_arr[x]]=not_detected[y_test_arr[x]]+1
        else:
          not_detected[y_test_arr[x]]=1

      else:
        false[yhat[x]]=1

        if y_test_arr[x] in not_detected:
          not_detected[y_test_arr[x]]=not_detected[y_test_arr[x]]+1
        else:
          not_detected[y_test_arr[x]]=1

In [0]:
true

{'BENIGN': 454281,
 'DoS Slowhttptest': 1064,
 'DoS slowloris': 1121,
 'PortScan': 31704,
 'Web Attack-Brute Force': 211,
 'Web Attack-XSS': 15}

In [0]:
false

{'BENIGN': 105,
 'DoS Slowhttptest': 49,
 'DoS slowloris': 31,
 'PortScan': 326,
 'Web Attack-Brute Force': 95,
 'Web Attack-XSS': 17}

In [0]:
not_detected

{'BENIGN': 400,
 'DoS Slowhttptest': 24,
 'DoS slowloris': 21,
 'PortScan': 33,
 'Web Attack-Brute Force': 45,
 'Web Attack-Sql Injection': 5,
 'Web Attack-XSS': 95}

## Deep Learning

In [36]:
import keras
from keras import backend as k
from keras.models import Sequential
from keras.layers import Activation

Using TensorFlow backend.


In [0]:
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

In [38]:
model=Sequential([
    Dense(32,input_shape=(16,), activation='relu'),
    Dense(64, activation='relu'),
    Dense(7, activation='softmax')
])

In [39]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 455       
Total params: 3,111
Trainable params: 3,111
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.compile(Adam(lr=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [0]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
def prepare_targets(y_train, y_test):
	le.fit(y_train)
	y_train_enc = le.transform(y_train)
	y_test_enc = le.transform(y_test)
	return y_train_enc, y_test_enc

In [0]:
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)

In [43]:
z=0
for i,j in zip(y_test, y_test_enc):
  print(i , " is encoded as :::: ", j)
  
  z=z+1
  if z>1500:
    break
  

BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
PortScan  is encoded as ::::  3
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  is encoded as ::::  0
BENIGN  

In [44]:
t1=time.time()


model.fit(X_train,y_train_enc,batch_size=20,epochs=20,shuffle=True,verbose=2)
t2=time.time()
print(" time taken for 20 epochs :: ", (t2-t1)/60 , " minutes")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/20





 - 73s - loss: 0.2045 - acc: 0.9024
Epoch 2/20
 - 72s - loss: 0.1538 - acc: 0.9326
Epoch 3/20
 - 71s - loss: 0.1064 - acc: 0.9717
Epoch 4/20
 - 73s - loss: 0.0711 - acc: 0.9820
Epoch 5/20
 - 72s - loss: 0.0554 - acc: 0.9876
Epoch 6/20
 - 71s - loss: 0.0464 - acc: 0.9914
Epoch 7/20
 - 71s - loss: 0.0407 - acc: 0.9930
Epoch 8/20
 - 72s - loss: 0.0369 - acc: 0.9936
Epoch 9/20
 - 71s - loss: 0.0344 - acc: 0.9939
Epoch 10/20
 - 70s - loss: 0.0326 - acc: 0.9941
Epoch 11/20
 - 71s - loss: 0.0312 - acc: 0.9942
Epoch 12/20
 - 71s - loss: 0.0301 - acc: 0.9943
Epoch 13/20
 - 70s - loss: 0.0292 - acc: 0.9944
Epoch 14/20
 - 70s - loss: 0.0284 - acc: 0.9945
Epoch 15/20
 - 70s - loss: 0.0277 - acc: 0.9946
Epoch 16/20
 - 70s - loss: 0.0269 - acc: 0.9947
Epoch 17/20
 - 72s - loss: 0.0264 - acc: 0.9948
Epoch 18/20
 - 70s - loss: 0.0258 - acc: 0.9949
Epoch 19/20
 - 71s - loss: 0.0254 - acc: 0.

In [0]:

# yhat=model.predict(X_test,batch_size=10,verbose=0) # It gives probability of classes
yhat=model.predict_classes(X_test,batch_size=10,verbose=0)

In [0]:
def calculate_confusion_matrix(y_test_arr,yhat):
  true={}
  false={}
  not_detected={}

  for x in range(len(y_test_arr)):
      if y_test_arr[x]==yhat[x]:
        if y_test_arr[x] in true:
          true[y_test_arr[x]]=true[y_test_arr[x]]+1
        else:
          true[y_test_arr[x]]=1
      elif y_test_arr[x]!=yhat[x]:
        if yhat[x] in false:
          false[yhat[x]]=false[yhat[x]]+1

          if y_test_arr[x] in not_detected:
            not_detected[y_test_arr[x]]=not_detected[y_test_arr[x]]+1
          else:
            not_detected[y_test_arr[x]]=1

        else:
          false[yhat[x]]=1

          if y_test_arr[x] in not_detected:
            not_detected[y_test_arr[x]]=not_detected[y_test_arr[x]]+1
          else:
            not_detected[y_test_arr[x]]=1
            
  print("========================================")
  print(" True values :: ", true)
  print(" False values :: ", false)
  print(" not detected values :: ", not_detected)    
  print("========================================")

In [47]:
  calculate_confusion_matrix(y_test_enc,yhat)

 True values ::  {0: 294347, 3: 31231, 1: 978, 2: 983, 4: 9}
 False values ::  {0: 1118, 3: 260, 2: 47, 1: 90}
 not detected values ::  {3: 439, 1: 100, 0: 374, 4: 285, 2: 178, 6: 133, 5: 6}
